<a href="https://colab.research.google.com/github/ustab/MY-GUIDED_PROJECTS_STUDIES/blob/main/NLP_code_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
download('stopwords')
download('wordnet')
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# task 1 - loading
data=pd.read_json('/content/recipes.json', lines=True)

In [3]:
data

,Name,url,Description,Author,Ingredients,Method
0,Christmas pie,https://www.bbcgoodfood.com/recipes/2793/chris...,Combine a few key Christmas flavours here to m...,Mary Cadogan,"[2 tbsp olive oil, knob butter, 1 onion, finel...",[Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp...
1,Simmer-&-stir Christmas cake,https://www.bbcgoodfood.com/recipes/1160/simme...,An easy-to-make alternative to traditional Chr...,Mary Cadogan,"[175g butter, chopped, 200g dark muscovado sug...","[Put the butter, sugar, fruit, zests, juice an..."
2,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/72622/chri...,These beautiful and classy little cakes make l...,Sara Buenfeld,"[200g dark muscovado sugar, 175g butter, chopp...","[Tip the sugar, butter, dried fruit, whole che..."
3,Christmas buns,https://www.bbcgoodfood.com/recipes/1803633/ch...,Paul Hollywood's fruit rolls can be made ahead...,Paul Hollywood,"[500g strong white flour, plus extra for dusti...",[Put the flour and 1 tsp salt into a large bow...
4,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/981634/chr...,"Made these for the second time today, and I ha...",Barney Desmazery,"[280g self-raising flour, 175g golden caster s...",[Heat oven to 190C/170 fan/gas 5 and line a 12...
...,...,...,...,...,...,...
1612,Flavoured butters,https://www.bbcgoodfood.com/recipes/1911/flavo...,Try these different flavoured butters melted o...,Maxine Clark,"[100g softened butter, 2-3 tbsp chopped fresh ...",[Take100g/4oz softened butter and beat until c...
1613,Crispy Roast Duck,https://www.bbcgoodfood.com/recipes/3248/crisp...,"Duck basted to perfection, ideal for a Sunday ...",Ruth Watson,"[3 medium-sized duckss (each about 2¼kg/5lb), ...",[Remove the duck’s giblets and wash them. Cut ...
1614,Smoked salmon kedgeree,https://www.bbcgoodfood.com/recipes/3047/smoke...,This kedgeree is totally delicious. Treat ever...,Barney Desmazery,"[300g basmati rice, 8 eggs, 400g pack sliced s...","[To make the sauce, heat the butter in a large..."
1615,Spicy potted crab,https://www.bbcgoodfood.com/recipes/1797/spicy...,An ideal elegant starter for a wedding,Merrilees Parker,"[6 x 250g pkts unsalted butter, 2 tsp hot chil...",[ Heat the butter in a pan until melted. Turn ...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         1617 non-null   object
 1   url          1617 non-null   object
 2   Description  1429 non-null   object
 3   Author       1611 non-null   object
 4   Ingredients  1617 non-null   object
 5   Method       1617 non-null   object
dtypes: object(6)
memory usage: 75.9+ KB


In [5]:
data['Ingredients'].head(5)

0    [2 tbsp olive oil, knob butter, 1 onion, finel...
1    [175g butter, chopped, 200g dark muscovado sug...
2    [200g dark muscovado sugar, 175g butter, chopp...
3    [500g strong white flour, plus extra for dusti...
4    [280g self-raising flour, 175g golden caster s...
Name: Ingredients, dtype: object

In [14]:
# task 2 - cleaning
data['Ingredients']=data['Ingredients'].map(lambda x: re.sub(r'[ˆa-zA-Z ]', ' ', str(x)))

In [15]:
data=data.dropna(subset=['Ingredients'])

In [18]:
stop=stopwords.words('english') + ['tsp', 'tbsp', 'finely', 'extra', 'chopped']

In [19]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
data.head(1)

,Name,url,Description,Author,Ingredients,Method
0,Christmas pie,https://www.bbcgoodfood.com/recipes/2793/chris...,Combine a few key Christmas flavours here to m...,Mary Cadogan,"['2 ', ' ', '1 , ...",[Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp...


In [36]:
#task3-remove encoding
def remove_encoding_word(word):
  word=str(word)
  word=word.encode('ASCII', 'Ignore').decode('ASCII')
  return word

In [43]:
def remove_encoding_text(text):
  text=str(text)
  text= ' '.join(remove_encoding_word(word) for word in text.split() if word not in stop)
  return text

In [ ]:
# task 4 - define lemmatizing
data['Ingredients'] = data['Ingredients'].apply(remove_encoding_text)

text= ' '.join(words for words in data['Ingredients'])
len(text)


In [45]:
lemma=WordNetLemmatizer().lemmatize

In [ ]:
lemma('leaves')

In [47]:
'leaf'

'leaf'

In [48]:
# task 5 - fit and transform text, with or without lemmatizing
def tokenize(document):
  tokens=[lemma(w) for w in document.split() if len(w)>3 and w.isalpha()]
  return tokens

In [55]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=((2,2)),
                            stop_words = stop, strip_accents = 'unicode')

In [ ]:
tdm= vectorizer.fit_transform(data['Ingredients'])

In [ ]:
vectorizer.vocabulary_.items()

In [ ]:
# task 6 -  get word frequencies and create wordcloud
tfidf_weights=[(word, tdm.getcol(idx).sum*()) for word, idx in vectorizer.vocabulary_.items()]

In [ ]:
tfidf_weights[0:10]

In [ ]:
w=WordCloud(width=1500, height=1200, mode='RGBA', background_color='white',
            max_words=2000).fit_words(dict(tfidf_weights))

In [ ]:
plt.figure(figsize=(20,15))
plt.imshow(w)
plt.axis('off')
plt.savefig('recipes_wordcloud.png')